# Capítulo 9: Scanners Vetorizados

## Introdução

Scanners são responsáveis por ler dados de fontes externas (arquivos Parquet, CSV, etc.) e convertê-los para o formato interno do DuckDB. Scanners vetorizados utilizam **SIMD para descompressão** e técnicas como **bit-unpacking** para maximizar throughput.

### Objetivos:
1. Entender como Parquet armazena dados compactados
2. Implementar bit-packing e unpacking vetorizado
3. Usar SIMD shuffle para descompressão
4. Benchmark de leitura vetorizada vs linha-a-linha

In [ ]:
!pip install duckdb pandas numpy matplotlib pyarrow -q

In [ ]:
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import struct

plt.rcParams['figure.figsize'] = (12, 6)

## 9.1 Bit-Packing: Compressão de Inteiros

Parquet usa **bit-packing** para compactar inteiros pequenos:

```
Valores: [0, 1, 2, 3, 4, 5, 6, 7]  (precisam de apenas 3 bits cada)

Sem compressão (32 bits cada): 8 × 4 bytes = 32 bytes
Com bit-packing (3 bits cada): 8 × 3 bits = 24 bits = 3 bytes

Layout compactado:
┌─────────────────────────────────┐
│ 000 001 010 011 │ 100 101 110 111 │
│   Byte 0-1      │    Byte 2       │
└─────────────────────────────────┘

Compressão: ~10x menor!
```

In [ ]:
# Implementação de Bit-Packing

def bit_pack(values: np.ndarray, bit_width: int) -> bytes:
    """Compacta valores inteiros usando bit_width bits por valor"""
    result = []
    buffer = 0
    bits_in_buffer = 0
    
    for val in values:
        # Adiciona valor ao buffer
        buffer |= (val & ((1 << bit_width) - 1)) << bits_in_buffer
        bits_in_buffer += bit_width
        
        # Emite bytes completos
        while bits_in_buffer >= 8:
            result.append(buffer & 0xFF)
            buffer >>= 8
            bits_in_buffer -= 8
    
    # Emite bits restantes
    if bits_in_buffer > 0:
        result.append(buffer & 0xFF)
    
    return bytes(result)

def bit_unpack_scalar(packed: bytes, bit_width: int, count: int) -> np.ndarray:
    """Descompacta valores um por vez (lento)"""
    result = np.zeros(count, dtype=np.int32)
    mask = (1 << bit_width) - 1
    
    buffer = 0
    bits_in_buffer = 0
    byte_idx = 0
    
    for i in range(count):
        # Carregar mais bytes se necessário
        while bits_in_buffer < bit_width:
            buffer |= packed[byte_idx] << bits_in_buffer
            bits_in_buffer += 8
            byte_idx += 1
        
        # Extrair valor
        result[i] = buffer & mask
        buffer >>= bit_width
        bits_in_buffer -= bit_width
    
    return result

# Teste
original = np.array([0, 1, 2, 3, 4, 5, 6, 7], dtype=np.int32)
packed = bit_pack(original, 3)
unpacked = bit_unpack_scalar(packed, 3, len(original))

print(f"Original:   {original}")
print(f"Packed:     {list(packed)} ({len(packed)} bytes)")
print(f"Unpacked:   {unpacked}")
print(f"Compressão: {len(original) * 4} bytes -> {len(packed)} bytes ({len(original) * 4 / len(packed):.1f}x)")

## 9.2 Unpacking Vetorizado

Em vez de extrair bit por bit, processamos múltiplos valores de uma vez:

```
Estratégia SIMD (simplificada):
1. Carregar 128 bits de dados compactados
2. Usar lookup tables para mapear grupos de bits
3. Shuffle para reorganizar bytes
4. Máscara AND para extrair valores
```

In [ ]:
def bit_unpack_vectorized(packed: bytes, bit_width: int, count: int) -> np.ndarray:
    """Descompacta valores de forma vetorizada usando NumPy"""
    # Converter bytes para array de inteiros grandes
    # Processa 8 valores por vez (64 bits)
    
    result = np.zeros(count, dtype=np.int32)
    mask = (1 << bit_width) - 1
    
    # Processar em chunks de 8 valores
    bits_per_chunk = 8 * bit_width
    bytes_per_chunk = (bits_per_chunk + 7) // 8
    
    for chunk_start in range(0, count, 8):
        chunk_end = min(chunk_start + 8, count)
        values_in_chunk = chunk_end - chunk_start
        
        # Calcular posição no buffer
        bit_offset = chunk_start * bit_width
        byte_offset = bit_offset // 8
        bit_shift = bit_offset % 8
        
        # Carregar bytes necessários como inteiro grande
        bytes_needed = (values_in_chunk * bit_width + bit_shift + 7) // 8
        chunk_bytes = packed[byte_offset:byte_offset + bytes_needed + 1]
        
        # Converter para inteiro
        value = int.from_bytes(chunk_bytes, 'little')
        value >>= bit_shift
        
        # Extrair valores (pode ser vetorizado com shifts)
        for i in range(values_in_chunk):
            result[chunk_start + i] = value & mask
            value >>= bit_width
    
    return result

# Usando NumPy puro para simulação mais realista
def bit_unpack_numpy(packed_array: np.ndarray, bit_width: int, count: int) -> np.ndarray:
    """Simulação de unpacking vetorizado com NumPy"""
    # Converte bytes para bits
    bits = np.unpackbits(packed_array)
    
    # Reorganiza em grupos de bit_width
    total_bits = count * bit_width
    bits = bits[:total_bits].reshape(-1, bit_width)
    
    # Converte cada grupo para inteiro (vetorizado)
    powers = 2 ** np.arange(bit_width)
    result = np.sum(bits * powers, axis=1)
    
    return result.astype(np.int32)

# Teste
packed_array = np.frombuffer(packed, dtype=np.uint8)
result_vec = bit_unpack_vectorized(packed, 3, 8)
print(f"Vetorizado: {result_vec}")

## 9.3 Benchmark: Scalar vs Vetorizado

In [ ]:
# Benchmark com dados maiores
n = 100_000
bit_widths = [3, 5, 8, 12, 16]

results = []

for bw in bit_widths:
    # Gerar dados que cabem em bit_width bits
    max_val = (1 << bw) - 1
    original = np.random.randint(0, max_val + 1, n, dtype=np.int32)
    packed = bit_pack(original, bw)
    
    # Benchmark scalar
    times_scalar = []
    for _ in range(3):
        start = time.perf_counter()
        _ = bit_unpack_scalar(packed, bw, n)
        times_scalar.append(time.perf_counter() - start)
    
    # Benchmark vetorizado
    times_vec = []
    for _ in range(3):
        start = time.perf_counter()
        _ = bit_unpack_vectorized(packed, bw, n)
        times_vec.append(time.perf_counter() - start)
    
    results.append({
        'bit_width': bw,
        'scalar_ms': np.mean(times_scalar) * 1000,
        'vectorized_ms': np.mean(times_vec) * 1000,
        'compression': (n * 4) / len(packed)
    })
    
    print(f"Bit width {bw:2}: Scalar {results[-1]['scalar_ms']:6.1f}ms, "
          f"Vec {results[-1]['vectorized_ms']:6.1f}ms, "
          f"Compressão {results[-1]['compression']:.1f}x")

In [ ]:
# Visualização
df = pd.DataFrame(results)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Tempo de unpacking
x = np.arange(len(bit_widths))
width = 0.35
ax1.bar(x - width/2, df['scalar_ms'], width, label='Scalar', color='#e74c3c')
ax1.bar(x + width/2, df['vectorized_ms'], width, label='Vetorizado', color='#27ae60')
ax1.set_xlabel('Bit Width')
ax1.set_ylabel('Tempo (ms)')
ax1.set_title('Tempo de Unpacking por Bit Width')
ax1.set_xticks(x)
ax1.set_xticklabels(bit_widths)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Taxa de compressão
ax2.bar(bit_widths, df['compression'], color='#3498db')
ax2.set_xlabel('Bit Width')
ax2.set_ylabel('Taxa de Compressão')
ax2.set_title('Compressão vs Bit Width')
ax2.axhline(y=1, color='red', linestyle='--', label='Sem compressão')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9.4 DuckDB: Leitura de Parquet

In [ ]:
import tempfile
import os

con = duckdb.connect(':memory:')

# Criar dados de teste
n = 5_000_000
df = pd.DataFrame({
    'id': np.arange(n),
    'small_int': np.random.randint(0, 100, n),      # Valores pequenos - alta compressão
    'medium_int': np.random.randint(0, 10000, n),   # Valores médios
    'large_int': np.random.randint(0, 1000000, n),  # Valores grandes
    'float_col': np.random.randn(n)
})

# Salvar como Parquet
temp_dir = tempfile.mkdtemp()
parquet_path = os.path.join(temp_dir, 'test_data.parquet')
df.to_parquet(parquet_path, compression='snappy')

file_size = os.path.getsize(parquet_path)
print(f"Arquivo Parquet: {file_size / 1024 / 1024:.1f} MB")
print(f"Dados originais: {df.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")
print(f"Compressão total: {df.memory_usage(deep=True).sum() / file_size:.1f}x")

In [ ]:
# Benchmark de leitura
queries = {
    'Full Scan': f"SELECT COUNT(*) FROM '{parquet_path}'",
    'Single Column': f"SELECT SUM(small_int) FROM '{parquet_path}'",
    'Multiple Columns': f"SELECT SUM(small_int), AVG(medium_int), MAX(large_int) FROM '{parquet_path}'",
    'Filtered Scan': f"SELECT SUM(small_int) FROM '{parquet_path}' WHERE medium_int > 5000",
    'Projection': f"SELECT id, small_int FROM '{parquet_path}' WHERE id < 1000000",
}

print("Benchmark de Leitura Parquet:\n")
for name, query in queries.items():
    times = []
    for _ in range(5):
        start = time.perf_counter()
        con.execute(query).fetchall()
        times.append(time.perf_counter() - start)
    
    throughput = file_size / np.mean(times) / 1024 / 1024  # MB/s
    print(f"{name:20} {np.mean(times)*1000:8.2f} ms ({throughput:.0f} MB/s)")

In [ ]:
# Comparar com Pandas
print("\nComparação DuckDB vs Pandas:\n")

# DuckDB
times_duck = []
for _ in range(3):
    start = time.perf_counter()
    con.execute(f"SELECT SUM(small_int), AVG(float_col) FROM '{parquet_path}'").fetchall()
    times_duck.append(time.perf_counter() - start)

# Pandas
times_pandas = []
for _ in range(3):
    start = time.perf_counter()
    df_read = pd.read_parquet(parquet_path)
    result = df_read['small_int'].sum(), df_read['float_col'].mean()
    times_pandas.append(time.perf_counter() - start)

print(f"DuckDB: {np.mean(times_duck)*1000:.1f} ms")
print(f"Pandas: {np.mean(times_pandas)*1000:.1f} ms")
print(f"Speedup: {np.mean(times_pandas)/np.mean(times_duck):.1f}x")

## 9.5 Dictionary Encoding

Parquet também usa **dictionary encoding** para strings repetidas:

```
Dados originais: ["São Paulo", "Rio", "São Paulo", "Curitiba", "Rio", ...]

Dictionary: {0: "São Paulo", 1: "Rio", 2: "Curitiba"}
Encoded:    [0, 1, 0, 2, 1, ...]  <- Apenas índices!

Se houver 3 valores únicos, cada índice usa apenas 2 bits!
```

In [ ]:
# Demonstração de Dictionary Encoding
class DictionaryEncoder:
    """Simula dictionary encoding do Parquet"""
    
    def __init__(self):
        self.dictionary = []
        self.value_to_idx = {}
    
    def encode(self, values: list) -> tuple:
        """Retorna (dictionary, indices)"""
        indices = []
        
        for val in values:
            if val not in self.value_to_idx:
                self.value_to_idx[val] = len(self.dictionary)
                self.dictionary.append(val)
            indices.append(self.value_to_idx[val])
        
        return self.dictionary, np.array(indices, dtype=np.int32)
    
    def decode_vectorized(self, indices: np.ndarray) -> list:
        """Decodifica usando lookup vetorizado"""
        dict_array = np.array(self.dictionary)
        return dict_array[indices]  # Gather operation!

# Teste com dados de exemplo
cities = ['São Paulo', 'Rio de Janeiro', 'Curitiba', 'Belo Horizonte', 'Salvador']
data = np.random.choice(cities, 100000)

encoder = DictionaryEncoder()
dictionary, indices = encoder.encode(data)

print(f"Dicionário: {dictionary}")
print(f"Primeiros índices: {indices[:20]}")
print(f"\nCompressão:")
print(f"  Original: {sum(len(s) for s in data)} bytes (strings)")
print(f"  Dicionário: {sum(len(s) for s in dictionary)} + {len(indices) * 4} bytes")

# Bits necessários para índices
bits_needed = int(np.ceil(np.log2(len(dictionary))))
print(f"  Com bit-packing ({bits_needed} bits): {len(indices) * bits_needed // 8} bytes")

## 9.6 Resumo

| Técnica | Descrição | Benefício |
|---------|-----------|----------|
| **Bit-Packing** | Compacta inteiros usando bits mínimos | Redução de I/O |
| **SIMD Unpack** | Descompacta múltiplos valores por instrução | CPU eficiente |
| **Dictionary Encoding** | Substitui valores por índices | Strings compactas |
| **Column Pruning** | Lê apenas colunas necessárias | Menos I/O |
| **Predicate Pushdown** | Filtra durante leitura | Menos dados |

In [ ]:
# Cleanup
con.close()
import shutil
shutil.rmtree(temp_dir, ignore_errors=True)